# Initial Orbit Determination with ALL Methods

This notebook performs Initial Orbit Determination (IOD) using several classical algorithms and Orekit's capabilities for orbital mechanics calculations. The main goal is to determine an initial orbit from three angular observations (e.g., Right Ascension and Declination) taken at different times. The code leverages Orekit's Java-based libraries and Python wrapper to access powerful astrodynamics tools.




Gauss initial orbit determination algorithm is given with this code. An initial orbit is determined from three angles.

*Reference: Vallado, D., Fundamentals of Astrodynamics and Applications*

In [4]:
# Initialize Orekit Virtual Machine (JVM) and set up environment
import orekit
orekit.initVM()  # Start Java Virtual Machine for Orekit functionality

# Set up Orekit resources and helper functions
from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()  # Prepare Orekit with current directory resources

# Import required Orekit and Hipparchus libraries
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.estimation.measurements import GroundStation, ObservableSatellite
from org.orekit.estimation.measurements import AngularRaDec
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint, CelestialBodyFactory
from org.orekit.estimation.iod import IodLambert, IodGibbs, IodLaplace, IodGooding
from org.orekit.time import TimeScalesFactory, AbsoluteDate, DateComponents, TimeComponents
from org.orekit.utils import IERSConventions, Constants, PVCoordinates, PVCoordinatesProvider, AbsolutePVCoordinates
from org.hipparchus.geometry.euclidean.threed import Vector3D, SphericalCoordinates
from org.orekit.data import DataProvidersManager, ZipJarCrawler
from org.orekit.orbits import KeplerianOrbit, CartesianOrbit, OrbitType
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.propagation import SpacecraftState

# Import additional standard Python libraries for mathematics, data handling, and plotting
from math import radians, pi, degrees, sin, cos, sqrt, acos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


First call the frames and time scales to be used in the code and create Earth body.

In [6]:
UTC = TimeScalesFactory.getUTC()                               # Define UTC time scale.
ECI = FramesFactory.getEME2000()                               # Define ECI reference frame.
GCRF = FramesFactory.getGCRF()
ICRF = FramesFactory.getICRF()
ECEF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)  # Define ECEF reference frame.
TEME = FramesFactory.getTEME()                                 # Define TEME reference frame. 
ITRF = ECEF

R_earth  = Constants.WGS84_EARTH_EQUATORIAL_RADIUS             # Radius of earth
Mu_earth = Constants.WGS84_EARTH_MU                            # Gravitational parameter of earth
f_earth  = Constants.WGS84_EARTH_FLATTENING                    # Earth flattening value

earth = OneAxisEllipsoid(R_earth, f_earth, ITRF)               # Create earth here.

Coordinates of the ground station that made the observations and its frame is entered below.

In [8]:
# Define the ground station in geodetic coordinates.
latitude = radians(40.0)
longitude = radians(-110.0)
altitude = 2000.0

# Frame of the ground station is given as Topocentric Frame.
station_coord = GeodeticPoint(latitude, longitude, altitude)
station_frame = TopocentricFrame(earth, station_coord, "Vallado")

# Create the ground station.
ground_station = GroundStation(station_frame)

# Create the observable satellite object, name it 1 as default
obs_sat = ObservableSatellite(1) 

### Gauss Algorithm

Gauss IOD algorithm accepts three observation angles and their respective observation dates as the input.

In [11]:
# Step 1: Define Observation Times and Positions
# Observation times are denoted as tau1, tau2, and tau3 for three successive observations.
# The positions are taken from angles converted to Cartesian coordinates, e.g., from Right Ascension and Declination.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Three seperate observations~~~~~~~~~~~~~~~~~~~~~~~~~
date1  = AbsoluteDate(2012, 8, 20, 11, 40, 28.0, UTC)             # Date of the observation
ra1    = radians(0.9399130)                                       # Observed right ascension 
dec1   = radians(18.667717)                                       # Observed declination

date2  = AbsoluteDate(2012, 8, 20, 11, 48, 28.0, UTC)
ra2    = radians(45.025748)
dec2   = radians(35.664741)

date3  = AbsoluteDate(2012, 8, 20, 11, 52, 28.0, UTC)
ra3    = radians(67.886655)
dec3   = radians(36.996583)

sigma  = [1.0, 1.0]
baseW  = [1.0, 1.0]

obsdata_df = pd.DataFrame(columns=['UTC Date', 'Ra (deg)', 'Dec (deg)'])


obsdata_df['Ra (deg)'] = [ra1, ra2, ra3]
obsdata_df['Dec (deg)'] = [dec1, dec2, dec3]
obsdata_df['UTC Date'] = [date1, date2, date3]


display(obsdata_df)

,UTC Date,Ra (deg),Dec (deg)
0,2012-08-20T11:40:28.000Z,0.016405,0.325813
1,2012-08-20T11:48:28.000Z,0.785848,0.622467
2,2012-08-20T11:52:28.000Z,1.184846,0.645712


In [12]:
# Step 2: Compute Line-of-Sight Vectors
# Convert angular observations to line-of-sight vectors for each observation time.
# These vectors are used to estimate the object's range from the observer.


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ First Observation ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
raDec1 = AngularRaDec(ground_station, ECI, date1, [ra1, dec1], 
                      sigma, baseW, obs_sat)
LOS1   = np.array([[cos(raDec1.getObservedValue()[1]) * cos(raDec1.getObservedValue()[0])],
                   [cos(raDec1.getObservedValue()[1]) * sin(raDec1.getObservedValue()[0])],
                   [sin(raDec1.getObservedValue()[1])]])

ground_station1_ECI = station_frame.getPVCoordinates(date1, ECI).getPosition()    
pos1 = np.array([[ground_station1_ECI.getX()], 
                 [ground_station1_ECI.getY()], 
                 [ground_station1_ECI.getZ()]])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Second Observation ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
raDec2 = AngularRaDec(ground_station, ECI, date2, [ra2, dec2], 
                      sigma, baseW, obs_sat)
LOS2   = np.array([[cos(raDec2.getObservedValue()[1]) * cos(raDec2.getObservedValue()[0])],
                   [cos(raDec2.getObservedValue()[1]) * sin(raDec2.getObservedValue()[0])],
                   [sin(raDec2.getObservedValue()[1])]])

ground_station2_ECI = station_frame.getPVCoordinates(date2, ECI).getPosition()    
pos2 = np.array([[ground_station2_ECI.getX()], 
                 [ground_station2_ECI.getY()], 
                 [ground_station2_ECI.getZ()]])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Third Observation ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
raDec3 = AngularRaDec(ground_station, ECI, date3, [ra3, dec3], 
                      sigma, baseW, obs_sat)
LOS3   = np.array([[cos(raDec3.getObservedValue()[1]) * cos(raDec3.getObservedValue()[0])],
                   [cos(raDec3.getObservedValue()[1]) * sin(raDec3.getObservedValue()[0])],
                   [sin(raDec3.getObservedValue()[1])]])

ground_station3_ECI = station_frame.getPVCoordinates(date3, ECI).getPosition()    
pos3 = np.array([[ground_station3_ECI.getX()], 
                 [ground_station3_ECI.getY()], 
                 [ground_station3_ECI.getZ()]])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
obs     = list([raDec1, raDec2, raDec3])
LOS     = np.hstack((LOS1, LOS2, LOS3))
LOS_inv = np.linalg.inv(LOS)
R_site  = np.hstack((pos1, pos2, pos3))



R_site_df = pd.DataFrame({'1st Observation': [R_site[0][0], R_site[1][0], R_site[2][0]],
                          '2nd Observation': [R_site[0][1], R_site[1][1], R_site[2][1]],
                          '3rd Observation': [R_site[0][2], R_site[1][2], R_site[2][2]]}, 
                           index = ['X (m)', 'Y (m)', 'Z (m)'])
R_site_df.index.name = 'R_site in ECI'
display(R_site_df)


tau1 = obs[0].getDate().durationFrom(obs[1].getDate())   # [sec]
tau3 = obs[2].getDate().durationFrom(obs[1].getDate())   # [sec]

a1   = tau3/(tau3 - tau1)
a1_u = tau3*((tau3 - tau1)**2 - tau3**2) / (6*(tau3 - tau1))
a3   = -tau1/(tau3 - tau1)
a3_u = -tau1*((tau3 - tau1)**2 - tau1**2) / (6*(tau3 - tau1))

M = np.matmul(LOS_inv, R_site)

d1 = M[1][0]*a1 - M[1][1] + M[1][2]*a3
d2 = M[1][0]*a1_u + M[1][2]*a3_u
C  = np.dot(LOS[:,1], R_site[:,1])

,1st Observation,2nd Observation,3rd Observation
R_site in ECI,,,
X (m),4.054677e+06,3.956010e+06,3.904854e+06
Y (m),2.748493e+06,2.888524e+06,2.957223e+06
Z (m),4.074238e+06,4.074365e+06,4.074431e+06


In [13]:
# Step 3: Estimate Range to Object
# Use an initial guess for the distance and iteratively refine it.
# This step calculates the distance from the observer to the object, which is essential for determining accurate position vectors.

#### Create an 8th degree polynomial in r_2
P = list(range(9))

P[0] = 1                                                   # 8th
P[1] = 0                                                   # 7th
P[2] = -(d1**2 + 2*C*d1 + np.linalg.norm(R_site[:,1])**2)  # 6th
P[3] = 0                                                   # 5th
P[4] = 0                                                   # 4th
P[5] = -2*Mu_earth*(C*d2 + d1*d2)                          # 3rd
P[6] = 0                                                   # 2nd
P[7] = 0                                                   # 1st
P[8] = -Mu_earth**2 * d2**2                                # 0th

roots = np.roots(P)

PosRoots = []
for item in roots:
    if np.isreal(item) and item > 0:
        PosRoots.append(item)

if len(PosRoots) == 1:
    r2 = PosRoots[0].real
else:
    pass

# Below value allows you to find an initial estimate of the slant-range values, 
# which ends Gauss’s part of the solution. First, find the three c_i coefficients.
u = Mu_earth / r2**3

c1 = -(-a1 -a1_u*u)
c2 = -1
c3 = -(-a3 -a3_u*u)

# Now, determine the initial guess of the slant ranges. Be sure to multiply the matrices out first.
B = np.matmul(M, np.array([[-c1], [-c2], [-c3]]))

A = np.identity(3)
A[0,0] = c1
A[1,1] = c2
A[2,2] = c3

slant_ranges = np.linalg.solve(A,B)
slant_ranges_df = pd.DataFrame(slant_ranges, 
                               index = ['1st Observation', '2nd Observation', '3rd Observation'])
slant_ranges_df.columns = ['Slant Range (m)']
display(slant_ranges_df)

position = []
for i in range(3):
    pos = slant_ranges[i]*LOS[:, i] + R_site[:, i]
    pos = np.transpose(pos) 
    position = np.concatenate((position, pos))

position = position.reshape((3, 3))
position_df = pd.DataFrame({'1st Observation': [position[0][0], position[1][0], position[2][0]],
                            '2nd Observation': [position[0][1], position[1][1], position[2][1]],
                            '3rd Observation': [position[0][2], position[1][2], position[2][2]]}, 
                            index = ['X (m)', 'Y (m)', 'Z (m)'])

display(position_df)

,Slant Range (m)
1st Observation,4.169430e+06
2nd Observation,4.104670e+06
3rd Observation,4.546328e+06


,1st Observation,2nd Observation,3rd Observation
X (m),8.004225e+06,2.813290e+06,5.408786e+06
Y (m),6.313015e+06,5.247649e+06,6.467557e+06
Z (m),5.271716e+06,6.321150e+06,6.810263e+06


### Refinement of Gauss IOD 

Lagrange coefficients *f* and *g* are calculated below to find the velocity vector *v_2*

In [16]:
# This section implements Steps 9–13 from Howard D. Curtis's "Orbital Mechanics for Engineering Students" (Algorithm 5.5).
# These steps guide us in determining the velocity vector v2 from the Gauss slant range solution.
# 
# The process includes:
# - Applying the Gauss method to calculate the slant range, which estimates the object's distance at the time of observation.
# - Using this distance to derive accurate position and velocity vectors, specifically at the second observation (t2).
# - Calculating the velocity vector v2 by leveraging position vectors and detailed stepwise approach in Algorithm 5.5.
#
# Algorithm is tailored for initial orbit determination with minimal observations, providing a reliable framework 
# for obtaining accurate orbital parameters with short observation arcs.

##################################################################
###### Curtis Algorithm 5.5 Steps 9-13 Continued
###### Finding the v2 Vector from Gauss Slant Range Solution
##################################################################


f1 = 1 - (0.5*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau1**2)
f3 = 1 - (0.5*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau3**2)
g1 = tau1 - ((1/6)*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau1**3)
g3 = tau3 - ((1/6)*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau3**3)

v2 = (1/(f1*g3 - f3*g1))*(-f3*position[0,:] + f1*position[2,:])

# Position vector p2 is taken from the result of Gauss IOD.
p2 = Vector3D(float(position[1,0]), float(position[1,1]), float(position[1,2]))
v2 = Vector3D(float(v2[0]), float(v2[1]), float(v2[2]))
PVcoord = PVCoordinates(p2, v2)

cartesianOrbit = CartesianOrbit(PVcoord, ECI, date2, Mu_earth)
keplerElements = KeplerianOrbit(cartesianOrbit.getPVCoordinates(date2, ECI), ECI, Constants.EGM96_EARTH_MU)



index_kepler = ['sma (m)', 'ecc', 'inc (deg)', 'aop (deg)', 'raan (deg)', 'ta (deg)']
refinedGauss_kepler_df = pd.DataFrame({'refinedGauss':[keplerElements.getA(),
                                                        keplerElements.getE(), 
                                                        degrees(keplerElements.getI()),
                                                        degrees(keplerElements.getPerigeeArgument()),
                                                        degrees(keplerElements.getRightAscensionOfAscendingNode()), 
                                                        degrees(keplerElements.getTrueAnomaly())]}, index = index_kepler)


index_cartesian = ['X (m)', 'Y (m)', 'Z (m)', 'Vx (m/s)', 'Vy (m/s)', 'Vz (m/s)']
refinedGauss_cartesian_df = pd.DataFrame({'refinedGauss':[PVcoord.getPosition().getX(),
                                                          PVcoord.getPosition().getY(), 
                                                          PVcoord.getPosition().getZ(),
                                                          PVcoord.getVelocity().getX(),
                                                          PVcoord.getVelocity().getY(),
                                                          PVcoord.getVelocity().getZ()]},index = index_cartesian)


## Circularised Gauss Method
Alternatively, one can assume the orbit is circular and do the following calculations to find the velocity of the satellite at the second observation time.

In [18]:
# This code implements the Circular Orbit Method for Initial Orbit Determination (IOD) using angle measurements.
# It is based on the thesis:
# 
# "Initial Orbit Determination Using Angle Measurements: Comparing IOD methods on very short arc observations of GEO objects
# from the MeerLICHT telescope" by Bert Van den Abbeele, 
#

##################################################################
###### Initial Orbit Determination Using Angle Measurements Thesis
###### 2.2.4. Circularised Gauss
##################################################################

f1 = 1 - (0.5*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau1**2)
f3 = 1 - (0.5*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau3**2)

g1 = tau1 - ((1/6)*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau1**3)
g3 = tau3 - ((1/6)*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau3**3)

v2 = (1/(f1*g3 - f3*g1))*(-f3*position[0,:] + f1*position[2,:])

#### If eccentricity is below 0.1, we may assume a circular orbit and do the following calculation.
h = np.cross(position[1,:], v2)
v_circGauss_unit = np.cross(h, position[1,:])/np.linalg.norm(np.cross(h, position[1,:]))
v_circGauss_mag = sqrt(Mu_earth/np.linalg.norm(position[1,:]))
v_circGauss_vector = v_circGauss_mag*v_circGauss_unit
print('\033[1m'+'\n Velocity vector with gaussian circularisation assumption \n'+'\033[0m',v_circGauss_vector)

# Position vector p2 is taken from the result of Gauss IOD.
p2 = Vector3D(float(position[1,0]), float(position[1,1]), float(position[1,2]))
v2 = Vector3D(float(v_circGauss_vector[0]), float(v_circGauss_vector[1]), float(v_circGauss_vector[2]))
PVcoord = PVCoordinates(p2, v2)

cartesianOrbit = CartesianOrbit(PVcoord, ECI, date2, Mu_earth)
keplerElements = KeplerianOrbit(cartesianOrbit.getPVCoordinates(date2, ECI), ECI, Constants.EGM96_EARTH_MU)




circularisedGauss_kepler_df = pd.DataFrame({'circularisedGauss':[keplerElements.getA(),
                                                                 keplerElements.getE(), 
                                                                 degrees(keplerElements.getI()),
                                                                 degrees(keplerElements.getPerigeeArgument()),
                                                                 degrees(keplerElements.getRightAscensionOfAscendingNode()), 
                                                                 degrees(keplerElements.getTrueAnomaly())]}, index = index_kepler)



circularisedGauss_cartesian_df = pd.DataFrame({'circularisedGauss':[PVcoord.getPosition().getX(),
                                                                    PVcoord.getPosition().getY(), 
                                                                    PVcoord.getPosition().getZ(),
                                                                    PVcoord.getVelocity().getX(),
                                                                    PVcoord.getVelocity().getY(),
                                                                    PVcoord.getVelocity().getZ()]},index = index_cartesian)


 Velocity vector with gaussian circularisation assumption 
 [-4502.47438686  4087.10916089  1078.68804582]


In [19]:
# This code implements the Circular Orbit Method for Initial Orbit Determination (IOD) using angle measurements.
# It is based on the thesis:
# 
# "Initial Orbit Determination Using Angle Measurements: Comparing IOD methods on very short arc observations of GEO objects
# from the MeerLICHT telescope" by Bert Van den Abbeele, 
#

##################################################################
###### Initial Orbit Determination Using Angle Measurements Thesis
###### 2.4. Circular Orbit Method
##################################################################

# Calculate time difference between observations
dt = tau3 - tau1

# Calculate dot product between two position vectors
dotproduct = np.dot(position[0, :], position[1, :])

# Initial guess for the optimization, based on the norm of the first position vector
assumption = np.linalg.norm(position[0, :])

from scipy.optimize import fsolve
import math

# Define equations to solve for x in circular orbit assumptions
def equations(p):
    x = p[0]  # Extract single element from array to avoid deprecation warning
    return dt * sqrt(Mu_earth / x**3) - acos(dotproduct / x**2)

# Solve the equation using fsolve, with the initial estimate (assumption)
x = fsolve(equations, assumption)

# Calculate circular orbit velocity magnitude
v_circ_mag = sqrt(Mu_earth / x[0])  # Access the first element to avoid deprecation

# Calculate the unit vector in the direction of the circular velocity
h_norm = np.cross(position[0, :], position[1, :]) / np.linalg.norm(np.cross(position[0, :], position[1, :]))
v_circ_unit = np.cross(h_norm, position[1, :]) / x[0]  # Access first element explicitly

# Compute the circular orbit velocity vector
v_circ_vector = v_circ_unit * v_circ_mag

# Display the circular orbit velocity vector
print('\033[1m'+'\n Velocity vector with circular orbit assumption \n'+'\033[0m', v_circ_vector)

# Create PVCoordinates with updated velocity and position for orbit propagation
p2 = Vector3D(float(position[1, 0]), float(position[1, 1]), float(position[1, 2]))
v2 = Vector3D(float(v_circ_vector[0]), float(v_circ_vector[1]), float(v_circ_vector[2]))
PVcoord = PVCoordinates(p2, v2)

# Convert to Cartesian and Keplerian orbital elements
cartesianOrbit = CartesianOrbit(PVcoord, ECI, date2, Mu_earth)
keplerElements = KeplerianOrbit(cartesianOrbit.getPVCoordinates(date2, ECI), ECI, Constants.EGM96_EARTH_MU)

# Store the Keplerian elements in a DataFrame
circularGauss_kepler_df = pd.DataFrame({
    'circularGauss': [
        keplerElements.getA(),
        keplerElements.getE(),
        degrees(keplerElements.getI()),
        degrees(keplerElements.getPerigeeArgument()),
        degrees(keplerElements.getRightAscensionOfAscendingNode()),
        degrees(keplerElements.getTrueAnomaly())
    ]
}, index=index_kepler)

# Store the Cartesian coordinates in a DataFrame
circularGauss_cartesian_df = pd.DataFrame({
    'circularGauss': [
        PVcoord.getPosition().getX(),
        PVcoord.getPosition().getY(),
        PVcoord.getPosition().getZ(),
        PVcoord.getVelocity().getX(),
        PVcoord.getVelocity().getY(),
        PVcoord.getVelocity().getZ()
    ]
}, index=index_cartesian)


 Velocity vector with circular orbit assumption 
 [-4479.12338769  4065.91235344  1073.09369003]


# Lambert IOD

In [21]:
# Define the position vectors at two different times (date1 and date2) using the Vector3D class.
# These vectors represent the object's position in space at the observation times.
p1 = Vector3D([float(position[0][0]), float(position[0][1]), float(position[0][2])])
p2 = Vector3D([float(position[1][0]), float(position[1][1]), float(position[1][2])])

# Initialize the Lambert solver using Earth's gravitational parameter (Mu_earth).
lambert = IodLambert(Mu_earth)

# Estimate the orbit using Lambert's method, given the two position vectors (p1 and p2) and their respective times (date1 and date2).
# - ECI: Earth-Centered Inertial reference frame.
# - True: Indicates that the orbit is prograde.
# - 0: Specifies the number of revolutions (0 for a short arc between p1 and p2).
estimated_orbit = lambert.estimate(ECI, True, 0, p1, date1, p2, date2)

# Convert the estimated orbit into Keplerian elements to extract classical orbital parameters.
keplerElements = KeplerianOrbit(estimated_orbit)

# Store the Keplerian elements in a DataFrame for organized display
Lambert_kepler_df = pd.DataFrame({'Lambert': [
    keplerElements.getA(),                    # Semi-major axis
    keplerElements.getE(),                    # Eccentricity
    degrees(keplerElements.getI()),           # Inclination (converted to degrees)
    degrees(keplerElements.getPerigeeArgument()),  # Argument of Perigee (converted to degrees)
    degrees(keplerElements.getRightAscensionOfAscendingNode()),  # RAAN (converted to degrees)
    degrees(keplerElements.getTrueAnomaly())  # True Anomaly (converted to degrees)
]}, index=index_kepler)

# Store the Cartesian coordinates of the estimated orbit in a DataFrame for display
Lambert_cartesian_df = pd.DataFrame({'Lambert': [
    estimated_orbit.getPVCoordinates().getPosition().getX(),
    estimated_orbit.getPVCoordinates().getPosition().getY(),
    estimated_orbit.getPVCoordinates().getPosition().getZ(),
    estimated_orbit.getPVCoordinates().getVelocity().getX(),
    estimated_orbit.getPVCoordinates().getVelocity().getY(),
    estimated_orbit.getPVCoordinates().getVelocity().getZ()
]}, index=index_cartesian)


# Gibbs IOD

In [23]:
# Test extracted from "Fundamentals of Astrodynamics & Applications", 
# D. Vallado, 3rd ed., Chapter on Initial Orbit Determination, Example 7-3, p.457

# Define position vectors for three consecutive observations.
# Each position vector corresponds to the observed position of the object in space at three distinct observation times.
posR1 = Vector3D(float(position[0][0]), float(position[0][1]), float(position[0][2]))  # First observation position
posR2 = Vector3D(float(position[1][0]), float(position[1][1]), float(position[1][2]))  # Second observation position
posR3 = Vector3D(float(position[2][0]), float(position[2][1]), float(position[2][2]))  # Third observation position

# Corresponding observation dates are gathered from Gauss method previously.

# Initialize the Gibbs IOD Method with Earth's gravitational parameter (Mu_earth).
gibbs = IodGibbs(Mu_earth)

# Estimate the orbit using the Gibbs method with the provided position vectors and observation dates.
# - The Gibbs method computes the orbital parameters based on three position observations,
#   which allows for calculating the object's initial orbit without needing velocity observations.
estimated_orbit = gibbs.estimate(ECI, posR1, date1, posR2, date2, posR3, date3)

# Convert the estimated orbit into Keplerian elements for a classical orbital parameter representation.
keplerElements = KeplerianOrbit(estimated_orbit)

# Store the Keplerian elements in a DataFrame for organized display
Gibbs_kepler_df = pd.DataFrame({
    'Gibbs': [
        keplerElements.getA(),                   # Semi-major axis
        keplerElements.getE(),                   # Eccentricity
        degrees(keplerElements.getI()),          # Inclination (converted to degrees)
        degrees(keplerElements.getPerigeeArgument()),  # Argument of Perigee (degrees)
        degrees(keplerElements.getRightAscensionOfAscendingNode()),  # RAAN (degrees)
        degrees(keplerElements.getTrueAnomaly())  # True Anomaly (degrees)
    ]
}, index=index_kepler)

# Store the Cartesian coordinates (position and velocity) of the estimated orbit in a DataFrame for display
Gibbs_cartesian_df = pd.DataFrame({
    'Gibbs': [
        estimated_orbit.getPVCoordinates().getPosition().getX(),
        estimated_orbit.getPVCoordinates().getPosition().getY(),
        estimated_orbit.getPVCoordinates().getPosition().getZ(),
        estimated_orbit.getPVCoordinates().getVelocity().getX(),
        estimated_orbit.getPVCoordinates().getVelocity().getY(),
        estimated_orbit.getPVCoordinates().getVelocity().getZ()
    ]
}, index=index_cartesian)


# Laplace IOD

In [25]:
# Initialize Laplace IOD method with Earth's gravitational parameter (Constants.EGM96_EARTH_MU).
laplace = IodLaplace(Constants.EGM96_EARTH_MU)

# Estimate the orbit using three right ascension and declination measurements (raDec1, raDec2, raDec3).
# These measurements provide angular observations of the target object at three successive observation times.

# Calculate the position and velocity vector at the second observation time (raDec2) in the ECI frame.
# obsPva provides position, velocity, and acceleration (PVA) coordinates of the ground station's base frame at the time of the second observation.
obsPva = ground_station.getBaseFrame().getPVCoordinates(raDec2.getDate(), ECI)
estimated_orbit = laplace.estimate(ECI, raDec1, raDec2, raDec3)

# Convert the estimated orbit into Keplerian elements for interpretation of orbital parameters.
keplerElements = KeplerianOrbit(estimated_orbit.getPVCoordinates(date2, ECI), ECI, Constants.EGM96_EARTH_MU)

# Store the Keplerian elements in a DataFrame for organized display
Laplace_kepler_df = pd.DataFrame({
    'Laplace': [
        keplerElements.getA(),                   # Semi-major axis
        keplerElements.getE(),                   # Eccentricity
        degrees(keplerElements.getI()),          # Inclination (converted to degrees)
        degrees(keplerElements.getPerigeeArgument()),  # Argument of Perigee (degrees)
        degrees(keplerElements.getRightAscensionOfAscendingNode()),  # RAAN (degrees)
        degrees(keplerElements.getTrueAnomaly())  # True Anomaly (degrees)
    ]
}, index=index_kepler)

# Store the Cartesian coordinates (position and velocity) of the estimated orbit in a DataFrame for display
Laplace_cartesian_df = pd.DataFrame({
    'Laplace': [
        estimated_orbit.getPVCoordinates().getPosition().getX(),
        estimated_orbit.getPVCoordinates().getPosition().getY(),
        estimated_orbit.getPVCoordinates().getPosition().getZ(),
        estimated_orbit.getPVCoordinates().getVelocity().getX(),
        estimated_orbit.getPVCoordinates().getVelocity().getY(),
        estimated_orbit.getPVCoordinates().getVelocity().getZ()
    ]
}, index=index_cartesian)


# Gooding IOD 

In [27]:
# Initialize Gooding IOD method with Earth's gravitational parameter (Mu_earth).
gooding = IodGooding(Mu_earth)

# Provide initial range guesses (in meters) for the object at the first and third observation points.
# These are starting estimates for the object's distance from the observer.
rho1init = 650000.0  # Initial guess for range at the first observation point
rho3init = 650000.0  # Initial guess for range at the third observation point

# Estimate the orbit using the Gooding method.
# - The Gooding method refines the orbit estimate using three angular measurements (raDec1, raDec2, raDec3)
#   and initial guesses for the range (distance) of the object at the first and third observations.
# - This method iteratively adjusts the estimated ranges until the computed orbit aligns with the angular data.
estimated_orbit = gooding.estimate(ECI, raDec1, raDec2, raDec3, rho1init, rho3init)

# Convert the estimated orbit into Keplerian elements to interpret classical orbital parameters.
keplerElements = KeplerianOrbit(estimated_orbit)

# Store the Keplerian elements in a DataFrame for organized display
Gooding_kepler_df = pd.DataFrame({
    'Gooding': [
        keplerElements.getA(),                   # Semi-major axis
        keplerElements.getE(),                   # Eccentricity
        degrees(keplerElements.getI()),          # Inclination (converted to degrees)
        degrees(keplerElements.getPerigeeArgument()),  # Argument of Perigee (degrees)
        degrees(keplerElements.getRightAscensionOfAscendingNode()),  # RAAN (degrees)
        degrees(keplerElements.getTrueAnomaly())  # True Anomaly (degrees)
    ]
}, index=index_kepler)

# Store the Cartesian coordinates (position and velocity) of the estimated orbit in a DataFrame for display
Gooding_cartesian_df = pd.DataFrame({
    'Gooding': [
        estimated_orbit.getPVCoordinates().getPosition().getX(),
        estimated_orbit.getPVCoordinates().getPosition().getY(),
        estimated_orbit.getPVCoordinates().getPosition().getZ(),
        estimated_orbit.getPVCoordinates().getVelocity().getX(),
        estimated_orbit.getPVCoordinates().getVelocity().getY(),
        estimated_orbit.getPVCoordinates().getVelocity().getZ()
    ]
}, index=index_cartesian)


# Comparison of The Results

In [29]:
# Vallado, D., Fundamentals of Astrodynamics and Applications
# Example 7-2. Determining an Initial Orbit Using Angles Only 

# Given True Data 
p_true = Vector3D(6356486.034, 5290532.2578, 6511396.979)
v_true = Vector3D(-4172.948, 4776.550, 1720.271)

PVcoord_true = PVCoordinates(p_true, v_true)

cartesianOrbit = CartesianOrbit(PVcoord_true, ECI, date2, Mu_earth)
keplerElements = KeplerianOrbit(cartesianOrbit.getPVCoordinates(date2, ECI), ECI, Constants.EGM96_EARTH_MU)


true_kepler_df = pd.DataFrame({'True data':[keplerElements.getA(),
                                            keplerElements.getE(), 
                                            degrees(keplerElements.getI()),
                                            degrees(keplerElements.getPerigeeArgument()),
                                            degrees(keplerElements.getRightAscensionOfAscendingNode()), 
                                            degrees(keplerElements.getTrueAnomaly())]}, index = index_kepler)



true_cartesian_df = pd.DataFrame({'True data':[PVcoord_true.getPosition().getX(),
                                               PVcoord_true.getPosition().getY(), 
                                               PVcoord_true.getPosition().getZ(),
                                               PVcoord_true.getVelocity().getX(),
                                               PVcoord_true.getVelocity().getY(),
                                               PVcoord_true.getVelocity().getZ()]},index = index_cartesian)

In [30]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
# pd.reset_option('display.float_format')

# Combine all Keplerian DataFrames for comparison
comparison_kepler_df = pd.concat([Laplace_kepler_df, refinedGauss_kepler_df, circularisedGauss_kepler_df, 
                                  circularGauss_kepler_df, Gooding_kepler_df, Gibbs_kepler_df, 
                                  Lambert_kepler_df, true_kepler_df], axis=1)

# Define a formatting function for bold styling
def make_bold(value):
    return 'font-weight: bold'

# Apply the formatting function using Styler.map
comparison_kepler_df = comparison_kepler_df.style.map(make_bold, subset=pd.IndexSlice[:, ['True data']])

# Display the styled DataFrame
display(comparison_kepler_df)


,Laplace,refinedGauss,circularisedGauss,circularGauss,Gooding,Gibbs,Lambert,True data
sma (m),10231201.262187,11548845.845749,10450898.182811,10343884.275269,3714709.572158,12125151.162004,11548910.397833,12246023.778360
ecc,0.056651,0.171883,0.000000,0.010346,0.989735,0.197727,0.166925,0.200000
inc (deg),40.071625,40.017990,40.017990,40.017990,41.567615,40.017990,40.017990,40.000007
aop (deg),-53.211703,9.154154,74.238866,254.238861,251.894469,19.746748,10.658540,20.000011
raan (deg),-29.547165,-30.034631,-30.034631,-30.034631,-29.684155,-30.034631,-30.034631,-30.000010
ta (deg),127.114824,65.084707,-0.000005,-180.000000,-179.451532,54.492113,46.060271,54.235979


The table compares the results of various Initial Orbit Determination (IOD) methods against the True data for a satellite's orbital parameters, including semi-major axis (sma), eccentricity (ecc), inclination (inc), argument of perigee (aop), right ascension of ascending node (raan), and true anomaly (ta). Here’s a breakdown of the accuracy and differences between each method and the true data:

**Semi-Major Axis (sma):**
The size of the orbit within the plane (its semi-major axis) can be estimated by solving for the object’s position and velocity vectors at multiple points. However, the accuracy here can be limited if range (distance) information is imprecise, as some methods like Laplace and Gooding rely heavily on angular data without direct range measurements.

The true semi-major axis is approximately 12246023.78 meters.
The Lambert and refined Gauss methods are relatively close to this value, showing errors within a few hundred thousand meters, while the Laplace and Gooding methods have larger deviations.
The Gibbs and circular methods also show significant discrepancies, indicating that these methods may be less suitable for this particular data set.

**Eccentricity (ecc):**
The orbit’s shape within the plane (its eccentricity) can be calculated using the three position vectors. For IOD methods that assume a particular orbital path (e.g., circular or near-circular), there may be limitations on eccentricity accuracy. 

The true eccentricity is 0.2.
Most methods yield values reasonably close to the true value, with Lambert and refined Gauss showing the best agreement.
The Gooding and Gibbs methods have notably higher eccentricities, indicating that they might have overestimated the orbit's shape for this data.

**Inclination (inc):**
Since inclination defines the tilt of the orbital plane relative to Earth’s equatorial plane, IOD methods are naturally well-suited to estimating it accurately. The inclination angle depends on the plane’s orientation in space, which IOD methods can calculate using three angular observations.

The true inclination is 40 degrees.
All methods produce inclination values close to the true value, with minimal differences, indicating they have correctly captured the orbit’s tilt relative to Earth’s equatorial plane.

**Argument of Perigee (aop):**
Argument of perigee is the orientation of the orbit within the plane, measured from the ascending node to the closest approach point (perigee). Accurate aop requires precise knowledge of the orbit’s shape, which can be challenging if there’s uncertainty in range or if the observations span a very short arc.

The true argument of perigee is around 20 degrees.
The Lambert and refined Gauss methods closely approximate this value, while other methods, like Gooding and Laplace, have larger deviations, suggesting that these methods might not capture the orientation of the orbit as accurately.

**Right Ascension of Ascending Node (raan):**
RAAN represents the orientation of the orbital plane with respect to a reference direction (e.g., the vernal equinox). Since IOD methods determine the plane’s orientation, they also estimate RAAN accurately as part of the orbital plane’s spatial positioning.

The true RAAN is -30 degrees.
Most methods, except for Laplace and Gooding, produce results close to the true RAAN, indicating generally good accuracy in determining the orbit’s orientation in the inertial frame.

**True Anomaly (ta):**
True anomaly indicates the position of the object within the orbital plane at a given time. IOD methods can estimate this accurately if the time between observations is sufficient, as it relies on calculating the object’s position along the elliptical path in the plane.

The true anomaly is 54.24 degrees.
Lambert and refined Gauss provide the closest estimates to the true anomaly, while other methods, particularly the circular and Gooding methods, diverge significantly.

In [31]:
# Combine all Cartesian DataFrames for comparison
comparison_cartesian_df = pd.concat([Laplace_cartesian_df, refinedGauss_cartesian_df, circularisedGauss_cartesian_df, 
                                     circularGauss_cartesian_df, Gooding_cartesian_df, Gibbs_cartesian_df, 
                                     Lambert_cartesian_df, true_cartesian_df], axis=1)

# Define a formatting function to make text bold
def make_bold(value):
    return 'font-weight: bold'

# Apply the bold formatting to the 'True data' column using Styler.map
comparison_cartesian_df = comparison_cartesian_df.style.map(make_bold, subset=pd.IndexSlice[:, ['True data']])

# Display the styled DataFrame
display(comparison_cartesian_df)


,Laplace,refinedGauss,circularisedGauss,circularGauss,Gooding,Gibbs,Lambert,True data
X (m),6375507.848416,6313015.150107,6313015.150107,6313015.150107,4527993.662043,6313015.150107,8004225.086914,6356486.034000
Y (m),5310197.390561,5247648.500137,5247648.500137,5247648.500137,3461021.958430,5247648.500137,2813289.951828,5290532.257800
Z (m),6531009.452317,6467557.312974,6467557.312974,6467557.312974,4655130.657429,6467557.312974,5408786.039946,6511396.979000
Vx (m/s),-4244.587081,-4101.072019,-4502.474387,-4479.123388,-958.014916,-4185.295217,-2843.574625,-4172.948000
Vy (m/s),4117.403669,4699.295806,4087.109161,4065.912353,176.384533,4788.141146,5401.188413,4776.550000
Vz (m/s),1252.393149,1692.373222,1078.688046,1073.093690,-284.840741,1721.558743,2731.044662,1720.271000
